In [1]:
!pip install simpletransformers -q

     |████████████████████████████████| 215kB 18.8MB/s 
     |████████████████████████████████| 122kB 46.0MB/s 
     |████████████████████████████████| 2.1MB 53.7MB/s 
     |████████████████████████████████| 51kB 8.3MB/s 
     |████████████████████████████████| 8.2MB 34.0MB/s 
     |████████████████████████████████| 81kB 7.5MB/s 
     |████████████████████████████████| 3.3MB 51.5MB/s 
     |████████████████████████████████| 1.2MB 42.8MB/s 
     |████████████████████████████████| 2.1MB 46.4MB/s 
     |████████████████████████████████| 204kB 62.5MB/s 
     |████████████████████████████████| 901kB 52.7MB/s 
     |████████████████████████████████| 163kB 60.2MB/s 
     |████████████████████████████████| 112kB 55.1MB/s 
     |████████████████████████████████| 4.2MB 45.9MB/s 
     |████████████████████████████████| 81kB 12.4MB/s 
     |████████████████████████████████| 102kB 11.8MB/s 
     |████████████████████████████████| 133kB 56.1MB/s 
     |████████████████████████████████| 112kB 55.6MB/

In [19]:
from simpletransformers.classification import ClassificationArgs, ClassificationModel
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score

In [21]:
df = pd.read_excel('/content/Data_Train.xlsx')
test_df = pd.read_excel('/content/Data_Test.xlsx')

In [4]:
df.columns = ['text', 'labels']
df.head()

,text,labels
0,But the most painful was the huge reversal in ...,3
1,How formidable is the opposition alliance amon...,0
2,Most Asian currencies were trading lower today...,3
3,"If you want to answer any question, click on ‘...",1
4,"In global markets, gold prices edged up today ...",3


In [5]:
df['labels'].value_counts()

1    2772
2    1924
0    1686
3    1246
Name: labels, dtype: int64

In [6]:
train_df = df.iloc[:5736]
eval_df = df.iloc[5736:]

In [45]:
# Optional model configuration
model_args = ClassificationArgs(num_train_epochs=5, reprocess_input_data=True, overwrite_output_dir=True)

# Create a ClassificationModel
model = ClassificationModel("roberta", "roberta-base", num_labels=4, args=model_args)

# Train the model
model.train_model(train_df)

Downloading:   0%|          | 0.00/442 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Running Epoch 1 of 5:   0%|          | 0/717 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/717 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/717 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/717 [00:00<?, ?it/s]

(3585, 0.10247044195784737)

In [46]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

  0%|          | 0/1892 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/237 [00:00<?, ?it/s]

In [47]:
result

{'eval_loss': 0.15559060713365913, 'mcc': 0.9580153349686046}

In [48]:
preds = np.argmax(model_outputs, axis=1)

In [53]:
accuracy_score(eval_df['labels'], preds)

0.9693446088794926

In [29]:
predictions, raw_outputs = model.predict(test_df['STORY'].values.tolist())

  0%|          | 0/2748 [00:00<?, ?it/s]

  0%|          | 0/344 [00:00<?, ?it/s]

In [32]:
sub = pd.DataFrame({'SECTION': predictions})
print(sub.shape)
sub.head()

(2748, 1)


,SECTION
0,1
1,2
2,1
3,1
4,1


In [34]:
sub.to_csv('roberta.csv', index=False)